<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/Topic_Modeling_with_BERTopic_Reclame_aqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling with BERTopic - Reclame Aqui**

BERTopic is a topic modeling technique that leverages transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions 

Reference: (https://maartengr.github.io/BERTopic/index.html).

### **Enabling the GPU**

We will use the GPU provided by COLAB to accelarate our model training. To enable GPUs for the notebook:
1- Navigate to Edit -> Notebook Settings
2- Select GPU from the Hardware Accelerator drop-down

In [3]:
# verify if GPU is enable
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jan  5 20:00:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **Setup**

In [4]:
%%capture
!pip install pyspellchecker
!pip install bertopic
!pip install kaleido # for save BERTopic plots as image

In [5]:
import pandas as pd # for data manipulation
import os # for interacting with the operating system
import nltk # for natural language processing
import string # for string manipulation
import re # for for regular expressions
import matplotlib.pyplot as plt # for visualization
import spacy # for lemmatize portuguese text
from bertopic import BERTopic # for topic modeling
from spellchecker import SpellChecker # for spell check

In [6]:
# Install spacy pt_core_news_sm for portuguese text
!python -m spacy download pt_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 20.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [7]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
# Download dataset with stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
# Donwload datasets for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [10]:
# Donwload dependency need to stem portuguese text
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

### **Load data from [Github](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git)**

In [11]:
!git clone https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git

Cloning into 'Topic-Modeling-Reclame-Aqui'...
remote: Enumerating objects: 16250, done.
remote: Counting objects: 100% (2069/2069), done.
remote: Compressing objects: 100% (1806/1806), done.
remote: Total 16250 (delta 285), reused 1945 (delta 232), pack-reused 14181
Receiving objects: 100% (16250/16250), 119.05 MiB | 16.31 MiB/s, done.
Resolving deltas: 100% (1433/1433), done.
Checking out files: 100% (14806/14806), done.


In [12]:
# Change directory
%cd /content/Topic-Modeling-Reclame-Aqui 

# Update files from remote repository
!git pull 

# Return to work directory
%cd ..

# Check current directory
!pwd

/content/Topic-Modeling-Reclame-Aqui
Already up to date.
/content
/content


In [13]:
def read_data(path_csv, drop_duplicates = True, lower=True):
 
  # Use the read_csv method to read csv file
  df = pd.read_csv(path_csv)
  
  if drop_duplicates:
    # Read and return the CSV file using the read_csv method
    print(f"Shape before remove duplicates: {df.shape}")

    # Use the drop_duplicated method to drop duplicates rows
    df = df.drop_duplicates(subset="text")

    print(f"Shape after remove duplicates: {df.shape}")

    if lower:
      # apply the str.lower() method to each element in the dataframe
      df = df.applymap(str.lower)
    
     # join columns
    df.columns = ["title", "documents"] 

    # Use the replace() method to replace the string with an empty string
    df = df.replace(re.compile('\[editado pelo reclame aqui\]|editado pelo reclame aqui|Editado pelo Reclame Aqui'), '')
    df = df.replace(re.compile('\[casas bahia\]|Casa Bahia|Casas Bahia|casa bahia'), '')
    df = df.replace(re.compile('\[magazine luiza\]|Magazine luiza|Magazine Luiza| Magazine luizar|Magazine Luizar'), '')
    df = df.replace(re.compile('\[mercado livre\]|Mercado Livre|Mercado livre'), '')
    df = df.replace(re.compile('\[americana\]|Ameriacanas|ameriacanas'), '')

    # Drop the old index column
    df.reset_index(inplace = True, drop = True)
  return df

### **Preprocessing**

#### **Tokenization**

Tokenization aims to breaking text down into its component parts

In [14]:
WORD_TOKENIZER = nltk.tokenize.word_tokenize
def tokenize(text):
  tokens = [token.strip().lower() for token in WORD_TOKENIZER(text, language="portuguese")]
  pattern = r"\b\w+\s+\w+\b"
  filtered_words = [word for word in tokens if re.search(pattern, word)]
  return [token for token in tokens if token not in filtered_words]

#### **Stem** 

Stem the tokens. This step aims to remove morphological affixes and normalize to standardized stem forms

In [15]:
STEMMER = nltk.stem.RSLPStemmer()
def stem(tokens):
  return [STEMMER.stem(token) for token in tokens]

#### **Lemmatize**

Lemmatize the tokens. Retains more natural forms than stemming, but assumes all tokens nons unless tokens are passed as (word, pos) tuples. Note: nltk lemmatize does not suport portugues language

In [16]:
LEMMATIZER = nltk.WordNetLemmatizer()

def lemmatize(tokens):
  lemmas = []
  for token in tokens:
      if isinstance(token, str):
          # treats token like a noun
          lemmas.append(LEMMATIZER.lemmatize(token)) 
      else: 
          # assume a tuple of (word, pos)
          lemmas.append(LEMMATIZER.lemmatize(*token))
  return lemmas

**Lemmatize option for portuguese text**

In [17]:
# load portuguese model
nlp = spacy.load('pt_core_news_sm')

def lemmatize_pt(tokens):

  # Create a spaCy Doc object and apply the lemmatization
  doc = nlp(' '.join(tokens))

  # Return lemmatize
  return [token.lemma_ for token in doc]

#### **Remove stopwords**

Stop words are things like articles and conjunctions that usually do not offer a lot of value in an analysis.

In [18]:
def remove_stopwords(tokens, stopwords=None, custom_stop_words = None):

  if custom_stop_words is None:
    custom_stop_words = ['amazon', 'americanas', 'casas bahia', 'magazine luiza', 'shein', 'kabum',
                       'samsung', 'mercado livre', 'banco brasil', 'apple', 'eletrolux', 'magazine', 'luiza', 'luizar',
                      'casas', 'bahia', 'casa', 'mercado', 'livre']

  # Use the default stop words if none is passed
  if stopwords is None:
    stopwords = nltk.corpus.stopwords.words('portuguese') + custom_stop_words
  
  # Filter the list of tokens to exclude the stop word tokens
  return [token for token in tokens if token not in stopwords]

In [19]:
custom_stop_words = ['amazon', 'americanas', 'casas bahia', 'magazine luiza', 'shein', 'kabum',
                       'samsung', 'mercado livre', 'banco brasil', 'apple', 'eletrolux', 'magazine', 'luiza', 'luizar',
                      'casas', 'bahia', 'casa', 'mercado', 'livre']
assert remove_stopwords(['compra', 'echar', 'em esse', 'amazon', 'pude'], custom_stop_words=custom_stop_words) == ['compra', 'echar', 'em esse', 'pude']

#### **Remove hyperlinks**

Removes http/s links from the tokens.

In [20]:
def remove_links(tokens):
  # Filter tokens that starts with "http://" or "https://"
  return [token for token in tokens 
          if not token.startswith("http://")
          and not token.startswith("https://")]

In [21]:
assert remove_links(['bom', 'http://online', 'https://offline']) == ['bom']

#### **Remove numbers**

In [22]:
def remove_numbers(tokens):
  # Use a regular expression to match words that contain numbers
  pattern = r"\b\w*\d\w*\b"
  tokens = [token for token in tokens if not re.sub(pattern, "", token) == ""]
  
  # Filter out number tokens using a list comprehension and the isnumeric method
  return [token for token in tokens if not token.isnumeric()]

In [23]:
assert remove_numbers(['ola', 'bicicleta', '1', '2002']) == ['ola', 'bicicleta']

#### **Remove date**

In [24]:
def remove_date(tokens):
  # Compile a regular expression to match dates in the format dd/mm or dd/mm/yyyy
  date_regex = re.compile(r'\d{2}/\d{2}(/\d{4})?')

  # Use the regex to find all the tokens that match the date pattern
  dates = [token for token in tokens if date_regex.fullmatch(token)]

  # Filter the list of tokens to exclude the date tokens
  filtered_tokens = [token for token in tokens if token not in dates]

  # Return the filtered tokens
  return filtered_tokens

In [25]:
assert remove_date(['texto', 'data', '20/10', 'seguro', '02/09/2014']) == ['texto', 'data', 'seguro']

#### **Remove punctuation**

In [26]:
def remove_punctuation(tokens,
                       strip_mentions=True,
                       strip_hashtags=True):

    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]

    # Filter punctuation tokens
    tokens = [token.strip() for token in tokens if token not in string.punctuation]

    # Remove @ symbol from left side of tokens
    if strip_mentions:
        tokens = [t.lstrip(r"([!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~])\1+") for t in tokens]

    # Remove # symbol from left side of tokens
    if strip_hashtags:
        tokens = [t.lstrip(r"([!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~])\1+") for t in tokens]

    return tokens

In [27]:
assert remove_punctuation(['limpo', 'acento/  ///', 'simples???', 'onde', ',']) == ['limpo', 'acento', 'simples', 'onde']

#### **Remove short tokens**

In [28]:
def remove_short_tokens(tokens):
  # Filter the list of tokens to exclude tokens that are shorter than four letters
  filtered_tokens = [token for token in tokens if len(token) >= 4]

  # Return the filtered tokens
  return filtered_tokens

In [29]:
assert remove_short_tokens(['sair', 'um', 'correto', 'igual', 'oi', 'de', 'em']) == ['sair', 'correto', 'igual']

#### **Correction of spelling errors**

In [30]:
# Create a SpellChecker object
spell = SpellChecker(language='pt')

def check_spell_errors(text):
  result = []
  for token in text:
    # Correct the spelling errors in the text
    corrected_text = spell.correction(token)

    # If no correction is present user the original text
    if corrected_text == None:
      corrected_text =  token
  
    result.append(corrected_text)
  # Return the corrected text
  return result

#### **Remove extra white spaces**

In [31]:
def remove_whitespace(document):
    return  " ".join(document.split())

In [32]:
def preprocess(df):
  df['documents']= df['documents'].str.lower()
  df['documents']= df['documents'].apply(remove_whitespace)
  df['tokenize'] = df['documents'].apply(tokenize)
  #df['spell_error'] = df['tokenize'].apply(check_spell_errors)
  df['remove_links'] = df['tokenize'].apply(remove_links)
  df['remove_punctuation'] = df['remove_links'].apply(remove_punctuation)
  df['remove_numbers'] = df['remove_punctuation'].apply(remove_numbers)
  df['remove_date'] = df['remove_numbers'].apply(remove_date)
  df['remove_short_tokens'] = df['remove_date'].apply(remove_short_tokens)
  df['remove_stop_words'] = df['remove_short_tokens'].apply(remove_stopwords)
  df['lemmas'] = df['remove_stop_words'].apply(lemmatize_pt) 
  df['corpus'] = df['lemmas'].apply(lambda x: ' '.join(x))
  return df

In [34]:
if not os.path.exists('/content/Topic-Modeling-Reclame-Aqui/results/table/table.csv'):

  path_csv = "/content/Topic-Modeling-Reclame-Aqui/corpus.csv"
  
  # read data 
  df = read_data(path_csv)
  
  # preprocess data
  df = preprocess(df)

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/table/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  # Save the DataFrame to a CSV file
  df.to_csv(path + 'table.csv')

Shape before remove duplicates: (12760, 2)
Shape after remove duplicates: (10510, 2)


In [37]:
corpus = pd.read_csv('/content/Topic-Modeling-Reclame-Aqui/results/table/table.csv')['corpus']
print(corpus[0:3])

0    pesquisando bastante novo comprar resolver agu...
1    sinceramente decepcionar entrar contato procon...
2    cancelei plano antes terminar período testir g...
Name: corpus, dtype: object


## **Training a BERTopic Model**

The BERTopic algorithm has several advantages over other topic modeling algorithms. It is able to handle sparse data, it is scalable to large datasets, and it is able to learn topics that are not well-defined or are overlapping.

As our data language is portuguese we will going to set language to multilingual.

Create a new BERTopic model and train it. By default BERTopic use the paraphrase-multilingual-MiniLM-L12-v2 model for multi language documents. For others model check here [BERTopic sentence transformers](https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#sentence-transformers)

In [38]:
# Create a new BERTopic model using multilingual option
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

# Train model 
topics, probs = topic_model.fit_transform(corpus)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/329 [00:00<?, ?it/s]

2023-01-05 20:09:27,783 - BERTopic - Transformed documents to Embeddings
2023-01-05 20:10:02,372 - BERTopic - Reduced dimensionality
2023-01-05 20:10:06,792 - BERTopic - Clustered reduced embeddings


BERTopic works in three main steps: 


1.   Documents are first converted to numeric data. It extracts different embeddings based on the context of the word. For this, a sentence transformation model is used.
2.  Documents with similar topics are then grouped together forming clusters with similar topics. For this purpose, BERTopic uses the clustering algorithm UMAP to lower the dimensionality of the embeddings. Then the documents are clustered with the density-based algorithm HDBSCAN.
3. BERTopic extracts topics from clusters using a class-based TF-IDF score. This score gives the importance of each word in a cluster. Topics are then created based on the most important words measured by their C-TF-IDF score.

For more information check this link [BERTopic](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6)



### **Extracting Topics**

In [39]:
# Print the most frequent topics
freq = topic_model.get_topic_info()

# Show the top 5 most frequent topics
freq.head(5)

,Topic,Count,Name
0,-1,5171,-1_entregar_pedir_produto_compra
1,0,613,0_cartão_crédito_cobrança_cobrar
2,1,493,1_celular_aparelho_smartphone_comprar
3,2,385,2_dinheiro_reembolso_valor_recebi
4,3,257,3_whatsapp_contato_entregar_atendimento


The table above shows the five most freqeuente topics and the words present on it extract by BERTopic. -1 refers to all outliers and should be ignored.

In [40]:
# show the most frequent topic
topic_model.get_topic(0)

[('cartão', 0.055043277403953185),
 ('crédito', 0.02979349322344244),
 ('cobrança', 0.02153946577247617),
 ('cobrar', 0.01908109858728945),
 ('valor', 0.017324214144042543),
 ('fatura', 0.016869889599734397),
 ('compra', 0.01675435481895409),
 ('assinatura', 0.01573841088485844),
 ('pagamento', 0.014450931952660383),
 ('prime', 0.013226467445292277)]

**Note:** BERTopic is stocastich which means that the topics might differ across runs this is mostly due to the stocastisch nature of UMAP

#### **Save topic info table as CSV**

In [41]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/topic_info_tables/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)

# Save table as csv
freq.head(10).to_csv(path + 'topic_info_preprocessed_lemma.csv')

## **Visualization**

### **Intertopic Distance Map**

This graph shows the distance intertopic and help us understand the promixity of topics

In [86]:
fig = topic_model.visualize_topics(width=800, height=800)
fig

#### **Save intertopic distance map**

In [87]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/intertopic_distance_map/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "idm_preprocessed_lemma.png", format="png")
fig.write_html(path + "idm_preprocessed_lemma.html")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/intertopic_distance_map/idm_preprocessed_lemma.png?raw=true)

### **Visualize Topic Hierarchy**

The topics that were created can be hierarchically reduced. This visualization shows how the topics relate to one another.

In [78]:
fig = topic_model.visualize_hierarchy(top_n_topics=30, width=800, height=800)
fig

#### **Save Hierarchical Clustering**

In [81]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/hierarchical_clustering/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "hc_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/hierarchical_clustering/hc_preprocessed_lemma.png?raw=true)

### **Visualize Terms**

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation.

In [77]:
fig = topic_model.visualize_barchart(top_n_topics=12, width=300, height=300)
fig

#### **Save Top Word Scores Bar Chart**

In [82]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/top_words_scores/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "tws_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/top_words_scores/tws_preprocessed_lemma.png?raw=true)

### **Visualize Topic Similarity**

This plot shows a similarity matrix by simply applying cosine similarities through those topic embeddings generate by BERTopic through both c-TF-IDF and embeddings. This matrix indicate how similar certain topics are to each other.

In [49]:
fig = topic_model.visualize_heatmap(n_clusters=20, width=800, height=800)
fig

#### **Save Similarity Matrix**

In [83]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/similarity_matrix/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "sm_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/similarity_matrix/sm_preprocessed_lemma.png?raw=true)

### **Visualize Term Score Decline**

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added.

In [51]:
fig = topic_model.visualize_term_rank()
fig

#### **Save Term score decline per Topic**

In [52]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/term_score_decline_topic/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "tsdp_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/term_socore_decline_topic/tsdp_preprocessed_lemma.png?raw=true)

### **Term search**

In [53]:
# Find topics that contains blackfriday term
similar_topics, similarity = topic_model.find_topics("blackfriday", top_n=5)

# Show similar topics
similar_topics

[9, 70, 72, 14, 64]

In [59]:
# Show a specific topic
topic_model.get_topic(64)

[('previsão', 0.05966412466361346),
 ('prever', 0.04368307220734265),
 ('vaga', 0.03402297813439039),
 ('envio', 0.029453412564828682),
 ('prazo', 0.028679514603766866),
 ('entregar', 0.02686273896296797),
 ('esperar', 0.024257154926665303),
 ('grávida', 0.023180401514076805),
 ('fielmentegostar', 0.023180401514076805),
 ('comor', 0.023180401514076805)]

### **Hiperparameter optimization**

In [60]:
%%capture
!pip install optuna
!pip install hdbscan
!pip install umap-learn

In [61]:
import json
import optuna
from hdbscan import HDBSCAN
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import ast

In [88]:
def save_documents(model, corpus, trial, reduced):
  fig = topic_model.visualize_documents(corpus, hide_document_hover=True, hide_annotations=True, width=800, height=800)

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/documents_n_topics/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"document_n_topics_trial_{trial}.png", format="png")
  fig.write_html(path + f"document_n_topics_trial_{trial}.html")

In [89]:
def save_topics(model, trial, reduced=False):
  fig = model.visualize_topics(width=800, height=800)

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/intertopic_distance_map/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"intertopic_distance_map_trial_{trial}.png", format="png")
  fig.write_html(path + f"intertopic_distance_map_trial_{trial}.html")

In [90]:
def save_hierarchy(model, trial, reduced=False):
  fig = model.visualize_hierarchy(width=800, height=800)

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/hierarchical_clustering/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"hierarchical_clustering_trial_{trial}.png", format="png")
  fig.write_html(path + f"hierarchical_clustering_trial_{trial}.html")

In [91]:
def save_top_words_scores(model, trial, reduced=False):
  fig = model.visualize_barchart(top_n_topics=12, width=300, height=300)

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/top_words_scores/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"top_words_scores_trial_{trial}.png", format="png")
  fig.write_html(path + f"top_words_scores_trial_{trial}.html")

In [92]:
def save_similarity_matrix(model, trial, reduced=False):
  fig = model.visualize_heatmap()

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/similarity_matrix/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"similarity_matrix_trial_{trial}.png", format="png")
  fig.write_html(path + f"similarity_matrix_trial_{trial}.html")

In [93]:
def save_term_rank(model, trial, reduced=False):
  fig = model.visualize_term_rank(width=800, height=800)

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/term_score_decline_topic/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"term_score_trial_{trial}.png", format="png")
  fig.write_html(path + f"term_score_trial_{trial}.html")

In [94]:
def save_hyperparameters(trial_params, label):
  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/hyperparameters/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  with open(path + f"hyperparameters_trial_{label}.json", "w") as f:
    f.write(json.dumps(trial_params))


In [95]:
def optimizer(trial):

  # BERTopic hyperparameters
  top_n_words = trial.suggest_int('bertopic__top_n_words', 5, 10)
  n_gram_range = ast.literal_eval(trial.suggest_categorical('bertopic__n_gram_range', ['(1,1)', '(1,2)', '(1,3)']))
  min_topic_size = trial.suggest_int('bertopic__min_topic_size', 15, 20)
  diversity = trial.suggest_float('bertopic__diversity', 0.1, 0.9)
  outlier_threshold = trial.suggest_float('bertopic__outliers_threshold', 0.04, 0.09)
  nr_topics = trial.suggest_int('bertopic__nr_topics', 12, 20)

  # UMAP hyperparameters
  n_neighbors = trial.suggest_int('umap__n_neighbors', 10, 20)
  n_components = trial.suggest_int('umap__n_components', 2, 6)
  metric = trial.suggest_categorical('umap__metric', ['cosine', 'euclidean'])
  min_dist = trial.suggest_float('umap__min_dist', 0.1, 1.0)
  spread = trial.suggest_float('umap__spread', 0.1, 1.0)

  # HDBSCAN hyperprarameters
  min_cluster_size = trial.suggest_int('hdbscan__min_cluster_size', 10, 20)
  min_samples = trial.suggest_int('hdbscan__min_samples', 10, 20)
  cluster_selection_epsilon = trial.suggest_float('hdbscan__cluster_selection_epsilon', 0.1, 1.0)

  # CountVectorizer hyperparameters 
  max_features = trial.suggest_int('vectorizer__max_features', 3000, 6000)

  # reduce the impact of frequent words.
  ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

  stop_words = nltk.corpus.stopwords.words('portuguese') + custom_stop_words
  vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=n_gram_range, max_features=max_features)

  umap_model = UMAP(n_neighbors=n_neighbors, metric=metric, random_state=42)

  hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, prediction_data=True)

  # create a new BERTopic model using multilingual option
  model = BERTopic(language="multilingual", 
                         calculate_probabilities=True, 
                         verbose=False,
                         nr_topics='auto',
                         top_n_words=top_n_words,
                         n_gram_range=n_gram_range,
                         min_topic_size=min_topic_size,
                         diversity=diversity,
                         umap_model=umap_model,
                         hdbscan_model=hdbscan_model,
                         vectorizer_model=vectorizer_model,
                         ctfidf_model=ctfidf_model)
  
  label = trial.number
  params = trial.params

  # train model 
  topics, probs = model.fit_transform(corpus)

  # save plots
  save_topics(model, label, False)
  save_documents(model, corpus, label, False)
  save_hierarchy(model, label, False)
  ##save_similarity_matrix(model, label, False)
  save_term_rank(model, label, False)
  save_top_words_scores(model, label, False)

  # reduce outliers
  new_topics = model.reduce_outliers(corpus,
                                     topics, 
                                     probabilities=probs,
                                     strategy="probabilities",
                                     threshold=outlier_threshold)
  model.update_topics(corpus, topics=new_topics)

  # save plots after reducion
  save_topics(model, label, True)
  save_documents(model, corpus, label, True)
  save_hierarchy(model, label, True)
  ##save_similarity_matrix(model, label, True)
  save_term_rank(model, label, True)
  save_top_words_scores(model, label, True)

  # save hyperparameters
  save_hyperparameters(params, label)

  # save model
  path = '/content/Topic-Modeling-Reclame-Aqui/results/models/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  model.save(path + f"model_trial_{label}")

  return 0

In [ ]:
%%time
study = optuna.create_study(study_name='Bertopic')
study.optimize(optimizer, n_trials=50)

#### **Merge of similar topics**

In [97]:
# TODO: select the best model and merge similar topic and save final model

#topic_model = BERTopic.load("best_model")

In [98]:
##########33!rm -rf /content/Topic-Modeling-Reclame-Aqui/results

In [ ]:
! ssh-keygen -t rsa -b 4096
# Add github.com to our known hosts
! ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
# Restrict the key permissions, or else SSH will complain.
! chmod go-rwx /root/.ssh/rsa

In [ ]:
! cat /root/.ssh/id_rsa.pub

In [123]:
!git config --global user.email "mattheus_ribeiro@outlook.com"
!git config --global user.name "punkmic"

In [ ]:
!ssh -T git@github.com

In [ ]:
%cd /content/Topic-Modeling-Reclame-Aqui/

In [119]:
!git add ./results/table

In [ ]:
!git status

In [ ]:
!git commit -m "Adding preprocessed dataset"

In [126]:
!git push 

fatal: could not read Username for 'https://github.com': No such device or address
